In [8]:
from langgraph.graph import StateGraph , START, END
from langgraph.types import Command, interrupt
from typing import TypedDict
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

class State(TypedDict):
    value : str

def node_a(state : State):
    print ("Node A")
    return Command(goto = "node_b", update = {"value": state["value"]+ "a"})

def node_b(state : State):
    print ("Node B")
    human_response = interrupt ("Do you want to go to node c or node d? Type C/D")
    print ("Human response Value: ", human_response)

    if (human_response =="C"):
        return Command(goto="node_c", update={"value":state["value"]+ "b"})
    
    elif (human_response =="D"):
        return Command(goto= "node_d", update={"value":state["value"]+ "b"})
    

def node_c(state:State):
    print ("Node C")
    return Command(goto =END, update ={"value":state["value"]+ "c"})

def node_d(state:State):
    print ("Node D")
    return Command(goto =END, update ={"value":state["value"]+ "d"})




graph = StateGraph(State)

graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)
graph.add_node("node_c", node_c)
graph.add_node("node_d", node_d)

graph.set_entry_point("node_a")

app = graph.compile(checkpointer= memory)

cofig = {"configurable":{"thread_id":"1"}}

initialState = {"value":""}
final_result = app.invoke(initialState,cofig, stream_mode= "updates")
final_result


Node A
Node B


[{'node_a': {'value': 'a'}},
 {'__interrupt__': (Interrupt(value='Do you want to go to node c or node d? Type C/D', resumable=True, ns=['node_b:bf50d32e-7939-664f-383c-b34c21a2c313'], when='during'),)}]

In [9]:
print (app.get_state(cofig).next)

('node_b',)


In [10]:
second_result = app.invoke(Command(resume= "D"), config= cofig, stream_mode="updates")
second_result



Node B
Human response Value:  D
Node D


[{'node_b': {'value': 'ab'}}, {'node_d': {'value': 'abd'}}]